In [2]:
import os
import sys
sys.path.insert(0,'../')
from libs.ssl_data import *
from libs.ssl_model import *
from libs.ssl_utils import *
from libs.ssl_task import *
from libs import eeg_utils
from sklearn.model_selection import train_test_split
# Use these commands in the same cell to autoreload local library source code
# https://stackoverflow.com/questions/45807760/change-source-code-and-update-jupyter-notebook-but-retain-variables
%load_ext autoreload
%autoreload 2

In [18]:
all_ds = load_concat_dataset(path=f'data/ds005509_preprocessed', preload=False)

In [5]:
EEG = mne.io.read_raw_eeglab('/mnt/nemar/openneuro/ds005514/sub-NDARRU751ATE/eeg/sub-NDARRU751ATE_task-RestingState_eeg.set')

Reading /mnt/nemar/openneuro/ds005514/sub-NDARRU751ATE/eeg/sub-NDARRU751ATE_task-RestingState_eeg.set


/tmp/ipykernel_3684363/2003494304.py:1: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  EEG = mne.io.read_raw_eeglab('/mnt/nemar/openneuro/ds005514/sub-NDARRU751ATE/eeg/sub-NDARRU751ATE_task-RestingState_eeg.set')


In [7]:
EEG.annotations.description

array(['break cnt', 'resting_start', 'instructed_toOpenEyes',
       'instructed_toCloseEyes', 'instructed_toOpenEyes',
       'instructed_toCloseEyes', 'instructed_toOpenEyes',
       'instructed_toCloseEyes', 'instructed_toOpenEyes',
       'instructed_toCloseEyes', 'instructed_toOpenEyes',
       'instructed_toCloseEyes', 'instructed_toOpenEyes', 'boundary'],
      dtype='<U22')

In [9]:
events_from_annot, event_dict = mne.events_from_annotations(EEG)
print(event_dict)
print(events_from_annot[:5])

Used Annotations descriptions: ['boundary', 'break cnt', 'instructed_toCloseEyes', 'instructed_toOpenEyes', 'resting_start']
{'boundary': 1, 'break cnt': 2, 'instructed_toCloseEyes': 3, 'instructed_toOpenEyes': 4, 'resting_start': 5}
[[    0     0     2]
 [  969     0     5]
 [22210     0     4]
 [32210     0     3]
 [52211     0     4]]


In [12]:
epochs = mne.Epochs(EEG, events_from_annot, event_id=event_dict, tmin=-0.3, tmax=0.7)

Not setting metadata
14 matching events found
Setting baseline interval to [-0.3, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated


In [13]:
print(epochs.event_id)

{'boundary': 1, 'break cnt': 2, 'instructed_toCloseEyes': 3, 'instructed_toOpenEyes': 4, 'resting_start': 5}


In [14]:
print(epochs["instructed_toCloseEyes"])

<Epochs | 5 events (good & bad), -0.3 – 0.7 s (baseline -0.3 – 0 s), ~175 KiB, data not loaded,
 'instructed_toCloseEyes': 5>


In [ ]:
from braindecode.preprocessing.windowers import create_windows_from_events
sfreq = all_ds.datasets[0].raw.info['sfreq']
window_size_samples = int(1 * sfreq)
mapping = {  # We merge stages 3 and 4 following AASM standards.
    'instructed_toCloseEyes': 0,
}
windows_dataset = create_windows_from_events(
    all_ds, trial_start_offset_samples=0, trial_stop_offset_samples=0,
    window_size_samples=window_size_samples,
    window_stride_samples=window_size_samples, preload=True)

Used Annotations descriptions: ['instructed_toCloseEyes', 'instructed_toOpenEyes']


ValueError: Window size 250 exceeds trial duration (0) for too many trials (100.0%). Set accepted_bads_ratio to at least 1.0and restart training to be able to continue.